In [1]:
pip install gtfs-realtime-bindings==0.0.7

  Created wheel for gtfs-realtime-bindings: filename=gtfs_realtime_bindings-0.0.7-py3-none-any.whl size=9830 sha256=13e3397a091d2e73f8e07a6df418ce81341def50aef7842c0cc05a30be3f5e14
  Stored in directory: /root/.cache/pip/wheels/83/74/d8/f52d8153f87f34738597950f7c75dd15cfd0c88923dc15ede8
Successfully built gtfs-realtime-bindings


In [2]:
from google.transit import gtfs_realtime_pb2
import urllib.request as ur
import urllib.error as ue
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import zipfile
import folium
import io
import requests

In [3]:
# カラムのみに空のDataFrameを作成
cols = ["id", "route_id", "url", "header_text", "description_text"]
df_alert = pd.DataFrame(columns=cols)
print(df_alert.head())

cols = ["id", "trip_id", "latitude", "longitude", "current_stop_sequence", "timestamp", "vehicle_id"]
df_vehicle = pd.DataFrame(columns=cols)
print(df_vehicle.head())

Empty DataFrame
Columns: [id, route_id, url, header_text, description_text]
Index: []
Empty DataFrame
Columns: [id, trip_id, latitude, longitude, current_stop_sequence, timestamp, vehicle_id]
Index: []


運行データを取得する

In [4]:
file_url = "http://opendata.sagabus.info/saga-current.zip"
f = io.BytesIO()
r = requests.get(file_url)
f.write(r.content)

with zipfile.ZipFile(f) as z:
    with z.open('trips.txt') as r:
         df_trips = pd.read_csv(r)

In [6]:
feed_alert = gtfs_realtime_pb2.FeedMessage()
response = ur.urlopen('http://opendata.sagabus.info/alert.pb')
feed_alert.ParseFromString(response.read())

989

リアルタイムデータを取得する

In [9]:
feed_vehicle = gtfs_realtime_pb2.FeedMessage()
response = ur.urlopen('http://opendata.sagabus.info/vehicle.pb')
feed_vehicle.ParseFromString(response.read())

8869

運用データ（df_vheicle）に含まれる路線情報と、リアルタイムデータ（df_trips）に含まれる車両の現在値情報を使いたい（一つのファイルとしてMapboxに渡したい）ので、df_vheicleとdf_tripsを、'trip_id'でマージする。

In [11]:
df = pd.merge(df_vehicle, df_trips, on='trip_id', how='outer', indicator=True)
df_new = df[df['_merge']=='both']
df_new['symbol']='bus'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


ここでは、streamlitは使わない。
ここでは、Mapboxは、トークンを不要としないタイプの関数を利用する。

In [12]:
fig = px.scatter_mapbox(df_vehicle, 
                        lat="latitude", 
                        lon="longitude", 
                        hover_name="vehicle_id",                    
                        text= df_new['trip_headsign'],                      
                        zoom=10, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()